# Sofia Air Case - Team BG-USA

#### Team BG-USA Team Members:
- Sergey Vichev (serjvichev@gmail.com) datachat: sergeyvi4ev
- 

In [1]:
import pandas as pd
import numpy as np
import s3fs
import scipy
from datetime import datetime
import matplotlib as plt
import seaborn as sns
import bokeh
import plotly
import sklearn
import tensorflow
import keras
from math import radians, cos, sin, asin, sqrt

Using TensorFlow backend.


## Business Understanding

## Data Understanding

### Data Upload
Data consists of several files which are uploaded

In [44]:
fs = s3fs.S3FileSystem(anon=True)

In [97]:
# **LIST THE AVAILABLE DATATHON CASES**
fs.ls('datacases/gd-2019/sofia_air_2')

['datacases/gd-2019/sofia_air_2/atmosphere_profile_train.csv',
 'datacases/gd-2019/sofia_air_2/construction_sites.csv',
 'datacases/gd-2019/sofia_air_2/household_heating.csv',
 'datacases/gd-2019/sofia_air_2/industrial_pollution.csv',
 'datacases/gd-2019/sofia_air_2/sofia-air-case2-test-set.zip',
 'datacases/gd-2019/sofia_air_2/sofia-air-case2.zip',
 'datacases/gd-2019/sofia_air_2/sofia_topo.csv',
 'datacases/gd-2019/sofia_air_2/stations_data_train.csv',
 'datacases/gd-2019/sofia_air_2/weather_lbsf_20161101-20161130_IP_train.csv']

In [162]:
with fs.open('datacases/gd-2019/sofia_air_2/atmosphere_profile_train.csv', 'rb') as f:
    df_atmosphere = pd.read_csv(f)
with fs.open('datacases/gd-2019/sofia_air_2/construction_sites.csv', 'rb') as f:
    df_construtions = pd.read_csv(f)
with fs.open('datacases/gd-2019/sofia_air_2/household_heating.csv', 'rb', encoding = "ISO-8859-1") as f:
    df_household = pd.read_csv(f)
with fs.open('datacases/gd-2019/sofia_air_2/sofia_topo.csv', 'rb', encoding = "ISO-8859-1") as f:
    df_sofiatopo = pd.read_csv(f)
with fs.open('datacases/gd-2019/sofia_air_2/stations_data_train.csv', 'rb', encoding = "ISO-8859-1") as f:
    df_stations = pd.read_csv(f)
with fs.open('datacases/gd-2019/sofia_air_2/weather_lbsf_20161101-20161130_IP_train.csv', 'rb', encoding = "ISO-8859-1") as f:
    df_weather = pd.read_csv(f)
# from data
df_stations_loc = pd.read_csv("Data/stations_locations.csv")
df_industrial = pd.read_csv("Data/industrial_pollution.csv", encoding = 'ISO-8859-1')

## Atmosphere and Dispersion Model

In [163]:
df_atmosphere.head()

,Date,HGHT(m),TEMP(C)
0,2016-11-01,595,9.6
1,2016-11-01,663,7.6
2,2016-11-01,844,5.4
3,2016-11-01,1047,3.6
4,2016-11-01,1284,1.5


Atmoshpere table contains height and temperature for given day and for corresponding height.
In the following code:
- we filter out only the values of HGHT(m) less than 2000 meters
- calculate the temperature in K
- calculate t/z value

In [164]:
df_atmosphere = df_atmosphere.loc[df_atmosphere['HGHT(m)'] < 2000]
df_atmosphere['TEMP(K)'] =  df_atmosphere['TEMP(C)'] + 273
df_atmosphere['t/z'] = df_atmosphere['TEMP(K)']/df_atmosphere['HGHT(m)']/100

In [165]:
# define the Atmospheric stability class with vertical temperature gradient
df_atmosphere.loc[df_atmosphere['t/z'] <= -1.9, 'PCS'] = 'A'
df_atmosphere.loc[df_atmosphere['t/z'].between(-1.9, 1.7), 'PCS'] = 'B'
df_atmosphere.loc[df_atmosphere['t/z'].between(-1.7, -1.5), 'PCS'] = 'C'
df_atmosphere.loc[df_atmosphere['t/z'].between(-1.5, 0.5), 'PCS'] = 'D'
df_atmosphere.loc[df_atmosphere['t/z'].between(-0.5, 1.5), 'PCS'] = 'E'
df_atmosphere.loc[df_atmosphere['t/z'].between(1.5, 4), 'PCS'] = 'F'
df_atmosphere.loc[df_atmosphere['t/z'] >= 4, 'PCS'] = 'G'

In [166]:
# σy and σx values based on Pasquill stability:
# if less than 1km and if > than 1km
df_atmosphere.loc[df_atmosphere['PCS'] == 'A', 'a'] = 213
df_atmosphere.loc[df_atmosphere['PCS'] == 'A', 'c<1km'] = 440.8
df_atmosphere.loc[df_atmosphere['PCS'] == 'A', 'd<1km'] = 1.941
df_atmosphere.loc[df_atmosphere['PCS'] == 'A', 'f<1km'] = 9.27
df_atmosphere.loc[df_atmosphere['PCS'] == 'A', 'c>1km'] = 459.7
df_atmosphere.loc[df_atmosphere['PCS'] == 'A', 'd>1km'] = 2.094
df_atmosphere.loc[df_atmosphere['PCS'] == 'A', 'f>1km'] = -9.6

df_atmosphere.loc[df_atmosphere['PCS'] == 'B', 'a'] = 156
df_atmosphere.loc[df_atmosphere['PCS'] == 'B', 'c<1km'] = 106.6
df_atmosphere.loc[df_atmosphere['PCS'] == 'B', 'd<1km'] = 1.149
df_atmosphere.loc[df_atmosphere['PCS'] == 'B', 'f<1km'] = 3.3
df_atmosphere.loc[df_atmosphere['PCS'] == 'B', 'c>1km'] = 108.2
df_atmosphere.loc[df_atmosphere['PCS'] == 'B', 'd>1km'] = 1.098
df_atmosphere.loc[df_atmosphere['PCS'] == 'B', 'f>1km'] = 2

df_atmosphere.loc[df_atmosphere['PCS'] == 'C', 'a'] = 104
df_atmosphere.loc[df_atmosphere['PCS'] == 'C', 'c<1km'] = 61
df_atmosphere.loc[df_atmosphere['PCS'] == 'C', 'd<1km'] = 0.911
df_atmosphere.loc[df_atmosphere['PCS'] == 'C', 'f<1km'] = 0
df_atmosphere.loc[df_atmosphere['PCS'] == 'C', 'c>1km'] = 61
df_atmosphere.loc[df_atmosphere['PCS'] == 'C', 'd>1km'] = 0.911
df_atmosphere.loc[df_atmosphere['PCS'] == 'C', 'f>1km'] = 0

df_atmosphere.loc[df_atmosphere['PCS'] == 'D', 'a'] = 68
df_atmosphere.loc[df_atmosphere['PCS'] == 'D', 'c<1km'] = 33.2
df_atmosphere.loc[df_atmosphere['PCS'] == 'D', 'd<1km'] = 0.725
df_atmosphere.loc[df_atmosphere['PCS'] == 'D', 'f<1km'] = -1.7
df_atmosphere.loc[df_atmosphere['PCS'] == 'D', 'c>1km'] = 44.5
df_atmosphere.loc[df_atmosphere['PCS'] == 'D', 'd>1km'] = 0.516
df_atmosphere.loc[df_atmosphere['PCS'] == 'D', 'f>1km'] = -13

df_atmosphere.loc[df_atmosphere['PCS'] == 'E', 'a'] = 50.5
df_atmosphere.loc[df_atmosphere['PCS'] == 'E', 'c<1km'] = 22.8
df_atmosphere.loc[df_atmosphere['PCS'] == 'E', 'd<1km'] = 0.678
df_atmosphere.loc[df_atmosphere['PCS'] == 'E', 'f<1km'] = -1.3
df_atmosphere.loc[df_atmosphere['PCS'] == 'E', 'c>1km'] = 55.4
df_atmosphere.loc[df_atmosphere['PCS'] == 'E', 'd>1km'] = 0.305
df_atmosphere.loc[df_atmosphere['PCS'] == 'E', 'f>1km'] = -34.0

df_atmosphere.loc[df_atmosphere['PCS'] == 'F', 'a'] = 34
df_atmosphere.loc[df_atmosphere['PCS'] == 'F', 'c<1km'] = 14.35
df_atmosphere.loc[df_atmosphere['PCS'] == 'F', 'd<1km'] = 0.740
df_atmosphere.loc[df_atmosphere['PCS'] == 'F', 'f<1km'] = -0.35
df_atmosphere.loc[df_atmosphere['PCS'] == 'F', 'c>1km'] = 62.6
df_atmosphere.loc[df_atmosphere['PCS'] == 'F', 'd>1km'] = 0.18
df_atmosphere.loc[df_atmosphere['PCS'] == 'F', 'f>1km'] = -48.6

In [167]:
# grouping buy most frequent class in a separate table and save as 'df_pcs' dataframe
df_pcs = df_atmosphere.groupby(['Date','PCS']).agg(lambda x:x.value_counts().index[0])
df_pcs.reset_index(level=df_pcs.index.names, inplace=True)
df_pcs['Date'] = df_pcs['Date'].str.strip()
df_pcs.head()

,Date,PCS,HGHT(m),TEMP(C),TEMP(K),t/z,a,c<1km,d<1km,f<1km,c>1km,d>1km,f>1km
0,2016-11-01,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0
1,2016-11-02,E,1503,10.0,283.0,0.002257,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0
2,2016-11-03,E,1487,13.2,280.0,0.003618,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0
3,2016-11-04,E,1326,4.4,277.4,0.001808,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0
4,2016-11-05,E,1279,5.3,281.7,0.001841,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0


The calculation of the stability class gives E class for all dates. We automatically pick the most frequent class in case it is different.

### Stations

In [168]:
df_stations.head()

,Date,STA-BG0052A,STA-BG0050A,STA-BG0073A,STA-BG0040A
0,2016-11-01,692.88,823.44,624.00,876.240
1,2016-11-02,1632.96,1756.56,1516.56,2382.288
2,2016-11-03,953.28,978.48,1086.00,680.736
3,2016-11-04,545.52,631.44,888.24,613.200
4,2016-11-05,1420.08,1664.40,1617.12,1608.480


In [169]:
#regroup the stations dataset
df_stations_melt = pd.melt(df_stations, ['Date'], ['STA-BG0052A', 'STA-BG0050A', 'STA-BG0073A', 'STA-BG0040A'],
                      var_name='Station', value_name='PM10')
df_stations_melt.head()

,Date,Station,PM10
0,2016-11-01,STA-BG0052A,692.88
1,2016-11-02,STA-BG0052A,1632.96
2,2016-11-03,STA-BG0052A,953.28
3,2016-11-04,STA-BG0052A,545.52
4,2016-11-05,STA-BG0052A,1420.08


In [170]:
df_stations.head()

,Date,STA-BG0052A,STA-BG0050A,STA-BG0073A,STA-BG0040A
0,2016-11-01,692.88,823.44,624.00,876.240
1,2016-11-02,1632.96,1756.56,1516.56,2382.288
2,2016-11-03,953.28,978.48,1086.00,680.736
3,2016-11-04,545.52,631.44,888.24,613.200
4,2016-11-05,1420.08,1664.40,1617.12,1608.480


## Industrial

In [153]:
df_industrial.head()

,X*,Y*,m,t/y
0,"'""42°44''16.66""""N""'","'""23°14''28.82""""E""'",8.0,0.38
1,"'""42°39''46.01""""N""'","'""23°23''19.70""""E""'",15.0,0.03
2,"'""42°39''46.47""""N""'","'""23°23''19.27""""E""'",15.0,0.20
3,"'""42°39''46.70""""N""'","'""23°23''19.07""""E""'",15.0,0.96
4,"'""42°39''47.12""""N""'","'""23°23''21.30""""E""'",15.0,1.58


In [189]:
df_industrial['mg/d'] = df_industrial['t/y']*1000*1000*1000/365
df_industrial['mg/d'].head()

0    1.041096e+06
1    8.219178e+04
2    5.479452e+05
3    2.630137e+06
4    4.328767e+06
Name: mg/d, dtype: float64

In [197]:
# df = pd.DataFrame([])
# df.append()
df_stations.head()

,Date,STA-BG0052A,STA-BG0050A,STA-BG0073A,STA-BG0040A
0,2016-11-01,692.88,823.44,624.00,876.240
1,2016-11-02,1632.96,1756.56,1516.56,2382.288
2,2016-11-03,953.28,978.48,1086.00,680.736
3,2016-11-04,545.52,631.44,888.24,613.200
4,2016-11-05,1420.08,1664.40,1617.12,1608.480


In [218]:
df_industrial.tail()

,X*,Y*,m,t/y,Date,g/d,mg/d
66,"'""42°44''58.28""""N""'","'""23°15''12.03""""E""'",120.0,2.68,2016-11-07,7.342466e+06,7.342466e+06
67,"'""42°44''57.43""""N""'","'""23°15''11.47""""E""'",15.0,0.10,2016-11-08,2.739726e+05,2.739726e+05
68,"'""42°44''57.88""""N""'","'""23°15''11.37""""E""'",15.0,0.21,2016-11-09,5.753425e+05,5.753425e+05
69,"'""42°45''51.84""""N""'","'""23°26''15.90""""E""'",12.0,1.21,2016-11-10,3.315068e+06,3.315068e+06
70,"'""42°45''51.64""""N""'","'""23°26''16.16""""E""'",4.0,0.29,2016-11-11,7.945205e+05,7.945205e+05


In [ ]:
dataframe.loc[1]['Date'] = factoryRow['mg/d']

In [237]:
df = df_industrial


j = 0
dataframe = pd.DataFrame(columns= ['Date', 'mg/d', 'id'])
dataframe.set_index('mg/d')
for index, row in df_stations.iterrows():
    
    for factoryIndex, factoryRow in df_industrial.iterrows():
    #         print(row['Date'])
    #         print(factoryRow['mg/d'])
    #     dataframe['mg/d'] = factoryRow['mg/d']
        dataframe = dataframe.append(factoryRow)
#     dataframe['id'] = factoryIndex
dataframe.tail()
        

,Date,mg/d,id,X*,Y*,g/d,m,t/y
66,2016-11-07,7.342466e+06,NaN,"'""42°44''58.28""""N""'","'""23°15''12.03""""E""'",7.342466e+06,120.0,2.68
67,2016-11-08,2.739726e+05,NaN,"'""42°44''57.43""""N""'","'""23°15''11.47""""E""'",2.739726e+05,15.0,0.10
68,2016-11-09,5.753425e+05,NaN,"'""42°44''57.88""""N""'","'""23°15''11.37""""E""'",5.753425e+05,15.0,0.21
69,2016-11-10,3.315068e+06,NaN,"'""42°45''51.84""""N""'","'""23°26''15.90""""E""'",3.315068e+06,12.0,1.21
70,2016-11-11,7.945205e+05,NaN,"'""42°45''51.64""""N""'","'""23°26''16.16""""E""'",7.945205e+05,4.0,0.29


In [200]:
dataframe.head()

,Date,mg/d,id


In [222]:
dataframe.shape

(605, 3)

In [107]:
df_stations_merged = pd.merge(df_stations, df_pcs, how='inner', left_on='Date', right_on='Date')
df_stations_merged.head()

,Date,Station,PM10,PCS,HGHT(m),TEMP(C),TEMP(K),t/z,a,c<1km,d<1km,f<1km,c>1km,d>1km,f>1km
0,2016-11-01,STA-BG0052A,692.88,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0
1,2016-11-01,STA-BG0050A,823.44,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0
2,2016-11-01,STA-BG0073A,624.00,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0
3,2016-11-01,STA-BG0040A,876.24,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0
4,2016-11-02,STA-BG0052A,1632.96,E,1503,10.0,283.0,0.002257,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0


In [108]:
#new feature day of week
df_stations_merged['Date'] = pd.to_datetime(df_stations_merged['Date'])
df_stations_merged['day_of_week'] = df_stations_merged['Date'].dt.day_name()
df_stations_merged.head()

,Date,Station,PM10,PCS,HGHT(m),TEMP(C),TEMP(K),t/z,a,c<1km,d<1km,f<1km,c>1km,d>1km,f>1km,day_of_week
0,2016-11-01,STA-BG0052A,692.88,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0,Tuesday
1,2016-11-01,STA-BG0050A,823.44,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0,Tuesday
2,2016-11-01,STA-BG0073A,624.00,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0,Tuesday
3,2016-11-01,STA-BG0040A,876.24,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0,Tuesday
4,2016-11-02,STA-BG0052A,1632.96,E,1503,10.0,283.0,0.002257,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0,Wednesday


### Industrial distances

In [109]:
df_industrial.head()

,X*,Y*,m,t/y
0,"'""42°44''16.66""""N""'","'""23°14''28.82""""E""'",8.0,0.38
1,"'""42°39''46.01""""N""'","'""23°23''19.70""""E""'",15.0,0.03
2,"'""42°39''46.47""""N""'","'""23°23''19.27""""E""'",15.0,0.20
3,"'""42°39''46.70""""N""'","'""23°23''19.07""""E""'",15.0,0.96
4,"'""42°39''47.12""""N""'","'""23°23''21.30""""E""'",15.0,1.58


In [58]:
from math import cos, sqrt
fixedIndustrialDf = pd.read_csv("Data/industrial_coord.txt")

def get_distance(Lat1, Long1, Lat2, Long2):
    x = Lat2 - Lat1
    y = (Long2 - Long1)*cos((Lat2 + Lat1)*0.00872664626)  
    return 111.138*sqrt(x*x+y*y)



pd.options.mode.chained_assignment = None  # default='warn'

factoryDistanceDf = pd.DataFrame([['STA-BG0040A'], ['STA-BG0050A'], ['STA-BG0052A'], ['STA-BG0073A'] ], columns=['Name'])

for i in range(df_industrial.shape[0]):
    #columnName = 'dist' + str(i)
    #factoryDistanceDf[columnName] = ''
    xCol = 'x' + str(i)
    factoryDistanceDf[xCol] = ''
    yCol = 'y' + str(i)
    factoryDistanceDf[yCol] = ''
    
    
    for j in range(factoryDistanceDf.shape[0]):
        stationLat = df_stations_loc.iloc[j]['Latitude']
        stationLon = df_stations_loc.iloc[j]['Longitude']
        
        factoryLat = fixedIndustrialDf.iloc[i]['Lat']
        factoryLon = fixedIndustrialDf.iloc[i]['Lon']
        
        #distance = get_distance(stationLat, stationLon, factoryLat, factoryLon)
        
        x = get_distance(stationLat, 0, factoryLat, 0)
        y = get_distance(0, stationLon, 0, factoryLon)

        #print(distance)
        #factoryDistanceDf.loc[j][columnName] = distance
        factoryDistanceDf.loc[j][xCol] = x
        factoryDistanceDf.loc[j][yCol] = y

In [59]:
# calculated distances
factoryDistanceDf.head()

,Name,x0,y0,x1,y1,x2,y2,x3,y3,x4,...,x66,y66,x67,y67,x68,y68,x69,y69,x70,y70
0,STA-BG0040A,0.630054,7.73888,7.72536,8.65027,7.71116,8.63699,7.70406,8.63082,7.6911,...,1.91493,6.40492,1.88869,6.42221,1.90258,6.4253,3.56842,14.0899,3.56224,14.0979
1,STA-BG0050A,6.37967,6.16228,1.97575,10.2269,1.96155,10.2136,1.95445,10.2074,1.94148,...,7.66455,4.82832,7.6383,4.8456,7.6522,4.84869,9.31803,15.6665,9.31186,15.6745
2,STA-BG0052A,7.94116,17.6515,0.414261,1.26235,0.40006,1.27563,0.392959,1.2818,0.379993,...,9.22603,16.3175,9.19979,16.3348,9.21369,16.3379,10.8795,4.17723,10.8733,4.18526
3,STA-BG0073A,7.57562,3.00785,0.779794,13.3813,0.765593,13.368,0.758492,13.3619,0.745526,...,8.8605,1.67389,8.83426,1.69117,8.84815,1.69426,10.514,18.8209,10.5078,18.8289


In [60]:
df_ind_model = pd.merge(df_stations_merged,factoryDistanceDf, how='left', left_on='Station', right_on='Name')
df_ind_model.columns

Index(['Date', 'Station', 'PM10', 'PCS', 'HGHT(m)', 'TEMP(C)', 'TEMP(K)',
       't/z', 'a', 'c<1km',
       ...
       'x66', 'y66', 'x67', 'y67', 'x68', 'y68', 'x69', 'y69', 'x70', 'y70'],
      dtype='object', length=159)

In [61]:
def calculateSigmaY(x, a):
    #sigmaY = a*x^0.894
    sigmaY = a * pow(x, 0.894)
    return sigmaY
 
def calculateSigmaZ(x, c, d, f):
    #sigmaZ = cx^(d+f)
    sigmaZ = c * pow(x, d)+f
    return sigmaZ
 
def calculateSigmaZFromRow(row, index):
    sigmaZ = calculateSigmaZ(row['x' + str(index)], row['c>1km'], row['d>1km'], row['f>1km'])
    return sigmaZ
 
def calculateSigmaYFromRow(row, index):
    sigmaY = calculateSigmaY(row['y' + str(index)], row['a'])
    return sigmaY
 
for index in range(0, df_industrial.shape[0]):
    df_ind_model['sigmaZ' + str(index)] = df_ind_model.apply(lambda row: calculateSigmaZFromRow(row, index), axis=1)
    df_ind_model['sigmaY' + str(index)] = df_ind_model.apply(lambda row: calculateSigmaYFromRow(row, index), axis=1)

In [62]:
#Old 
#df_ind_model.head()

In [63]:
for column in df_ind_model:
    print(column)

Date
Station
PM10
PCS
HGHT(m)
TEMP(C)
TEMP(K)
t/z
a
c<1km
d<1km
f<1km
c>1km
d>1km
f>1km
day_of_week
Name
x0
y0
x1
y1
x2
y2
x3
y3
x4
y4
x5
y5
x6
y6
x7
y7
x8
y8
x9
y9
x10
y10
x11
y11
x12
y12
x13
y13
x14
y14
x15
y15
x16
y16
x17
y17
x18
y18
x19
y19
x20
y20
x21
y21
x22
y22
x23
y23
x24
y24
x25
y25
x26
y26
x27
y27
x28
y28
x29
y29
x30
y30
x31
y31
x32
y32
x33
y33
x34
y34
x35
y35
x36
y36
x37
y37
x38
y38
x39
y39
x40
y40
x41
y41
x42
y42
x43
y43
x44
y44
x45
y45
x46
y46
x47
y47
x48
y48
x49
y49
x50
y50
x51
y51
x52
y52
x53
y53
x54
y54
x55
y55
x56
y56
x57
y57
x58
y58
x59
y59
x60
y60
x61
y61
x62
y62
x63
y63
x64
y64
x65
y65
x66
y66
x67
y67
x68
y68
x69
y69
x70
y70
sigmaZ0
sigmaY0
sigmaZ1
sigmaY1
sigmaZ2
sigmaY2
sigmaZ3
sigmaY3
sigmaZ4
sigmaY4
sigmaZ5
sigmaY5
sigmaZ6
sigmaY6
sigmaZ7
sigmaY7
sigmaZ8
sigmaY8
sigmaZ9
sigmaY9
sigmaZ10
sigmaY10
sigmaZ11
sigmaY11
sigmaZ12
sigmaY12
sigmaZ13
sigmaY13
sigmaZ14
sigmaY14
sigmaZ15
sigmaY15
sigmaZ16
sigmaY16
sigmaZ17
sigmaY17
sigmaZ18
sigmaY18
sigmaZ19
sigmaY19
sigmaZ2

In [64]:
df_ind_model.columns

Index(['Date', 'Station', 'PM10', 'PCS', 'HGHT(m)', 'TEMP(C)', 'TEMP(K)',
       't/z', 'a', 'c<1km',
       ...
       'sigmaZ66', 'sigmaY66', 'sigmaZ67', 'sigmaY67', 'sigmaZ68', 'sigmaY68',
       'sigmaZ69', 'sigmaY69', 'sigmaZ70', 'sigmaY70'],
      dtype='object', length=301)

In [65]:
df_ind_model['PM10micro'] = df_ind_model['PM10'] / 1.225

In [66]:
# for indexheight in range(0,71):
#     df_ind_model['m'+str(indexheight)] = df_industrial.loc['m']

### Weather

In [67]:
df_weather.head()

,year,Month,day,TASMAX,TASAVG,TASMIN,DPMAX,DPAVG,DPMIN,RHMAX,...,sfcWindMAX,sfcWindAVG,sfcWindMIN,PSLMAX,PSLAVG,PSLMIN,PRCPMAX,PRCPAVG,PRCPMIN,VISIB
0,2016,11,1,12.7788,6.6672,0.0000,2.2224,-1.1112,-3.3336,87,...,24.14010,12.07005,0.0,1026.753044,1024.890530,1023.028016,-9999,0.00,-9999,5.471756
1,2016,11,2,15.5568,6.6672,-1.6668,2.7780,0.5556,-2.7780,100,...,11.26538,5.63269,0.0,1026.414405,1020.826864,1015.239322,-9999,0.00,-9999,8.046700
2,2016,11,3,13.3344,8.3340,3.3336,7.2228,3.3336,-1.1112,100,...,28.96812,14.48406,0.0,1023.366655,1019.133669,1014.900683,-9999,5.08,-9999,7.563898
3,2016,11,4,10.5564,6.1116,1.6668,6.1116,3.3336,1.1112,100,...,28.96812,14.48406,0.0,1025.398488,1023.197336,1020.996183,-9999,0.00,-9999,9.816974
4,2016,11,5,15.0012,8.8896,2.7780,10.0008,6.6672,2.7780,100,...,14.48406,7.24203,0.0,1024.043933,1020.657544,1017.271155,-9999,0.00,-9999,9.977908


In [68]:
def formatDateOfMonth(date):
    if len(str(date)) == 1:
        return '0'+ str(date)
    return date

df = df_weather
df['year'] = df['year'].astype(str)
df['Month'] = df['Month'].astype(str)
df['day'] = df['day'].apply(formatDateOfMonth)
df['day'] = df['day'].astype(str)

df['Date'] = df['year'].str.cat(df['Month'], sep='-')
df['Date'] = df['Date'].str.cat(df['day'], sep='-')#  +  + '-' + df['day']
df_weather = df

In [69]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 23 columns):
year          20 non-null object
Month         20 non-null object
day           20 non-null object
TASMAX        20 non-null float64
TASAVG        20 non-null float64
TASMIN        20 non-null float64
DPMAX         20 non-null float64
DPAVG         20 non-null float64
DPMIN         20 non-null float64
RHMAX         20 non-null int64
RHAVG         20 non-null float64
RHMIN         20 non-null int64
sfcWindMAX    20 non-null float64
sfcWindAVG    20 non-null float64
sfcWindMIN    20 non-null float64
PSLMAX        20 non-null float64
PSLAVG        20 non-null float64
PSLMIN        20 non-null float64
PRCPMAX       20 non-null int64
PRCPAVG       20 non-null float64
PRCPMIN       20 non-null int64
VISIB         20 non-null float64
Date          20 non-null object
dtypes: float64(15), int64(4), object(4)
memory usage: 3.7+ KB


In [70]:
df_industrial.head()

,X*,Y*,m,t/y
0,"'""42°44''16.66""""N""'","'""23°14''28.82""""E""'",8.0,0.38
1,"'""42°39''46.01""""N""'","'""23°23''19.70""""E""'",15.0,0.03
2,"'""42°39''46.47""""N""'","'""23°23''19.27""""E""'",15.0,0.20
3,"'""42°39''46.70""""N""'","'""23°23''19.07""""E""'",15.0,0.96
4,"'""42°39''47.12""""N""'","'""23°23''21.30""""E""'",15.0,1.58


### Final Merge

In [71]:
df_ind_model['Date'] = df_ind_model['Date'].astype(str)
df_ind_model['Date'] = df_ind_model['Date'].astype(str)

In [72]:
df_ind_model = pd.merge(df_ind_model,df_weather, how='left', left_on='Date', right_on='Date')
df_ind_model

,Date,Station,PM10,PCS,HGHT(m),TEMP(C),TEMP(K),t/z,a,c<1km,...,sfcWindMAX,sfcWindAVG,sfcWindMIN,PSLMAX,PSLAVG,PSLMIN,PRCPMAX,PRCPAVG,PRCPMIN,VISIB
0,2016-11-01,STA-BG0052A,692.880,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,24.14010,12.07005,0.00000,1026.753044,1024.890530,1023.028016,-9999,0.000,-9999,5.471756
1,2016-11-01,STA-BG0050A,823.440,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,24.14010,12.07005,0.00000,1026.753044,1024.890530,1023.028016,-9999,0.000,-9999,5.471756
2,2016-11-01,STA-BG0073A,624.000,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,24.14010,12.07005,0.00000,1026.753044,1024.890530,1023.028016,-9999,0.000,-9999,5.471756
3,2016-11-01,STA-BG0040A,876.240,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,24.14010,12.07005,0.00000,1026.753044,1024.890530,1023.028016,-9999,0.000,-9999,5.471756
4,2016-11-02,STA-BG0052A,1632.960,E,1503,10.0,283.0,0.002257,50.5,22.8,...,11.26538,5.63269,0.00000,1026.414405,1020.826864,1015.239322,-9999,0.000,-9999,8.046700
5,2016-11-02,STA-BG0050A,1756.560,E,1503,10.0,283.0,0.002257,50.5,22.8,...,11.26538,5.63269,0.00000,1026.414405,1020.826864,1015.239322,-9999,0.000,-9999,8.046700
6,2016-11-02,STA-BG0073A,1516.560,E,1503,10.0,283.0,0.002257,50.5,22.8,...,11.26538,5.63269,0.00000,1026.414405,1020.826864,1015.239322,-9999,0.000,-9999,8.046700
7,2016-11-02,STA-BG0040A,2382.288,E,1503,10.0,283.0,0.002257,50.5,22.8,...,11.26538,5.63269,0.00000,1026.414405,1020.826864,1015.239322,-9999,0.000,-9999,8.046700
8,2016-11-03,STA-BG0052A,953.280,E,1487,13.2,280.0,0.003618,50.5,22.8,...,28.96812,14.48406,0.00000,1023.366655,1019.133669,1014.900683,-9999,5.080,-9999,7.563898
9,2016-11-03,STA-BG0050A,978.480,E,1487,13.2,280.0,0.003618,50.5,22.8,...,28.96812,14.48406,0.00000,1023.366655,1019.133669,1014.900683,-9999,5.080,-9999,7.563898


In [73]:
list(df_ind_model.columns)

['Date',
 'Station',
 'PM10',
 'PCS',
 'HGHT(m)',
 'TEMP(C)',
 'TEMP(K)',
 't/z',
 'a',
 'c<1km',
 'd<1km',
 'f<1km',
 'c>1km',
 'd>1km',
 'f>1km',
 'day_of_week',
 'Name',
 'x0',
 'y0',
 'x1',
 'y1',
 'x2',
 'y2',
 'x3',
 'y3',
 'x4',
 'y4',
 'x5',
 'y5',
 'x6',
 'y6',
 'x7',
 'y7',
 'x8',
 'y8',
 'x9',
 'y9',
 'x10',
 'y10',
 'x11',
 'y11',
 'x12',
 'y12',
 'x13',
 'y13',
 'x14',
 'y14',
 'x15',
 'y15',
 'x16',
 'y16',
 'x17',
 'y17',
 'x18',
 'y18',
 'x19',
 'y19',
 'x20',
 'y20',
 'x21',
 'y21',
 'x22',
 'y22',
 'x23',
 'y23',
 'x24',
 'y24',
 'x25',
 'y25',
 'x26',
 'y26',
 'x27',
 'y27',
 'x28',
 'y28',
 'x29',
 'y29',
 'x30',
 'y30',
 'x31',
 'y31',
 'x32',
 'y32',
 'x33',
 'y33',
 'x34',
 'y34',
 'x35',
 'y35',
 'x36',
 'y36',
 'x37',
 'y37',
 'x38',
 'y38',
 'x39',
 'y39',
 'x40',
 'y40',
 'x41',
 'y41',
 'x42',
 'y42',
 'x43',
 'y43',
 'x44',
 'y44',
 'x45',
 'y45',
 'x46',
 'y46',
 'x47',
 'y47',
 'x48',
 'y48',
 'x49',
 'y49',
 'x50',
 'y50',
 'x51',
 'y51',
 'x52',
 'y52',

## Define and calculate C(x,y,z)

In [74]:
def C_xyz(Q,u,sigmaY,sigmaZ,h,z,y):
#    Cxyz = ((np.e**((-(z-h)**2)/(2*sigmaZ**2)))+(np.e**((-(z+h)**2)/(2*sigmaZ**2))))
    Cxyz = (Q/(2*np.pi*u*sigmaY*sigmaZ))*(np.e**((y**2)/(2*sigmaY**2)))*((np.e**((-(z-h)**2)/(2*sigmaZ**2)))+(np.e**((-(z+h)**2)/(2*sigmaZ**2))))
    return Cxyz

In [75]:
# list(df_ind_model.columns)

#### Assumptions:
- y=x becuase the pollutions spreads radially
- z = 0, becuase of the ground level
- h is the height of the chimney

In [76]:
def calculateCxyz(row, index):
    Cxyz = C_xyz(Q=row['PM10micro'],
                 u=row['sfcWindAVG'],
#                  sigmaY = 300,
                 sigmaY = row['sigmaY' + str(index)], 
#                  sigmaZ = 300,
                 sigmaZ = row['sigmaZ' + str(index)],
                 h = df_industrial.loc[index]['m'],
                 z = 0, 
                 y = row['y' + str(index)])
    return Cxyz

df_ind_model['Ctotal'] = 0
 
for index in range(0, df_industrial.shape[0]):
    df_ind_model['C' + str(index)] = df_ind_model.apply(lambda row: calculateCxyz(row, index), axis=1)
    df_ind_model['Ctotal'] = df_ind_model['Ctotal'] + df_ind_model['C' + str(index)]

In [77]:
df_ind_model

,Date,Station,PM10,PCS,HGHT(m),TEMP(C),TEMP(K),t/z,a,c<1km,...,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70
0,2016-11-01,STA-BG0052A,692.880,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,1.408322e-06,0.000837,6.001781e-04,0.000687,0.000678,9.044566e-05,0.000318,0.000318,0.001008,0.001017
1,2016-11-01,STA-BG0050A,823.440,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,1.673836e-05,0.000633,1.624575e-03,0.001884,0.001878,2.753297e-04,0.001212,0.001211,0.000393,0.000397
2,2016-11-01,STA-BG0073A,624.000,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,9.866119e-07,0.000337,6.499269e-04,0.000738,0.000734,6.060122e-04,0.002212,0.002207,0.000240,0.000242
3,2016-11-01,STA-BG0040A,876.240,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,1.338528e-04,0.001019,-2.335282e-172,-0.000021,-0.000010,3.518470e-06,0.001925,0.001917,0.000714,0.000734
4,2016-11-02,STA-BG0052A,1632.960,E,1503,10.0,283.0,0.002257,50.5,22.8,...,7.112345e-06,0.004227,3.031035e-03,0.003471,0.003423,4.567710e-04,0.001605,0.001604,0.005092,0.005135
5,2016-11-02,STA-BG0050A,1756.560,E,1503,10.0,283.0,0.002257,50.5,22.8,...,7.651334e-05,0.002891,7.426157e-03,0.008610,0.008582,1.258570e-03,0.005542,0.005535,0.001795,0.001815
6,2016-11-02,STA-BG0073A,1516.560,E,1503,10.0,283.0,0.002257,50.5,22.8,...,5.138242e-06,0.001755,3.384798e-03,0.003846,0.003825,3.156092e-03,0.011518,0.011491,0.001249,0.001262
7,2016-11-02,STA-BG0040A,2382.288,E,1503,10.0,283.0,0.002257,50.5,22.8,...,7.798158e-04,0.005938,-1.360516e-171,-0.000122,-0.000056,2.049832e-05,0.011212,0.011167,0.004157,0.004278
8,2016-11-03,STA-BG0052A,953.280,E,1487,13.2,280.0,0.003618,50.5,22.8,...,1.614668e-06,0.000960,6.881155e-04,0.000788,0.000777,1.036977e-04,0.000364,0.000364,0.001156,0.001166
9,2016-11-03,STA-BG0050A,978.480,E,1487,13.2,280.0,0.003618,50.5,22.8,...,1.657493e-05,0.000626,1.608713e-03,0.001865,0.001859,2.726414e-04,0.001201,0.001199,0.000389,0.000393


In [78]:
df_ind_model.Ctotal.sum()

15.902878906305947

In [37]:
df_construtions.head()

,id,start date,type,district,locality,address
0,100,07.1.2016,non-residential,OVCHA KUPEL,SEKULITSA,SUHOL
1,101,11.1.2016,non-residential,KRASNA POLYANA,TRUDOVI KAZARMI,street SUHOLSKA
2,102,11.1.2016,non-residential,MLADOST,MLADOST 2,NaN
3,103,13.1.2016,infrastructure,ISKAR,NADEZHDA 2A 2B,NaN
4,104,13.1.2016,infrastructure,NADEZHDA,STANKE DIMITROV,NaN


In [38]:
df_household.head()

,X,Y,NJ16_eq_1,NJ16_eq_2,NJ16_eq_3,NJ17_eq_1,NJ17_eq_3,NJ17_eq_4,NJ17_eq_6,NJ17_eq_7,NJ17_eq_4i,NJ17_eq_8,NJ17_eq_9,NN_Jilisht,NBROI_LICA
0,23.383978,42.693180,NaN,NaN,1.0,NaN,NaN,0,1,0,1,NaN,NaN,1,4
1,23.383978,42.693180,NaN,NaN,1.0,NaN,NaN,0,1,0,1,NaN,NaN,1,2
2,23.382120,42.693912,NaN,1.0,NaN,NaN,NaN,0,1,0,1,NaN,NaN,1,4
3,23.376602,42.678282,NaN,NaN,1.0,NaN,NaN,0,0,1,1,NaN,NaN,1,4
4,23.383978,42.693180,NaN,NaN,1.0,NaN,NaN,0,1,0,1,NaN,NaN,1,1


In [39]:
df_industrial.head()

,X*,Y*,m,t/y
0,"'""42°44''16.66""""N""'","'""23°14''28.82""""E""'",8.0,0.38
1,"'""42°39''46.01""""N""'","'""23°23''19.70""""E""'",15.0,0.03
2,"'""42°39''46.47""""N""'","'""23°23''19.27""""E""'",15.0,0.20
3,"'""42°39''46.70""""N""'","'""23°23''19.07""""E""'",15.0,0.96
4,"'""42°39''47.12""""N""'","'""23°23''21.30""""E""'",15.0,1.58


In [40]:
df_sofiatopo.head()

,Lat,Lon,Elev
0,42.62,23.220000,1184.0
1,42.62,23.233571,1333.0
2,42.62,23.247143,1505.0
3,42.62,23.260714,1586.0
4,42.62,23.274286,1533.0


In [41]:
df_stations.head()

,Date,Station,PM10
0,2016-11-01,STA-BG0052A,692.88
1,2016-11-02,STA-BG0052A,1632.96
2,2016-11-03,STA-BG0052A,953.28
3,2016-11-04,STA-BG0052A,545.52
4,2016-11-05,STA-BG0052A,1420.08


In [42]:
df_stations_loc.head()

,AirQualityStationEoICode,CommonName,Longitude,Latitude
0,BG0040A,Nadezhda,23.310972,42.732292
1,BG0050A,Hipodruma,23.296786,42.680558
2,BG0052A,Druzhba,23.400164,42.666508
3,BG0073A,IAOS/Pavlovo,23.268403,42.669797


### Stations

In [52]:
df_stations.head()

,Date,STA-BG0052A,STA-BG0050A,STA-BG0073A,STA-BG0040A
0,2016-11-01,692.88,823.44,624.00,876.240
1,2016-11-02,1632.96,1756.56,1516.56,2382.288
2,2016-11-03,953.28,978.48,1086.00,680.736
3,2016-11-04,545.52,631.44,888.24,613.200
4,2016-11-05,1420.08,1664.40,1617.12,1608.480


In [53]:
#regroup the stations dataset
df_stations = pd.melt(df_stations, ['Date'], ['STA-BG0052A', 'STA-BG0050A', 'STA-BG0073A', 'STA-BG0040A'],
                      var_name='Station', value_name='PM10')
df_stations.head()

,Date,Station,PM10
0,2016-11-01,STA-BG0052A,692.88
1,2016-11-02,STA-BG0052A,1632.96
2,2016-11-03,STA-BG0052A,953.28
3,2016-11-04,STA-BG0052A,545.52
4,2016-11-05,STA-BG0052A,1420.08


In [54]:
df_stations.head()

,Date,Station,PM10
0,2016-11-01,STA-BG0052A,692.88
1,2016-11-02,STA-BG0052A,1632.96
2,2016-11-03,STA-BG0052A,953.28
3,2016-11-04,STA-BG0052A,545.52
4,2016-11-05,STA-BG0052A,1420.08


In [55]:
df_stations_merged = pd.merge(df_stations, df_pcs, how='inner', left_on='Date', right_on='Date')
df_stations_merged.head()

,Date,Station,PM10,PCS,HGHT(m),TEMP(C),TEMP(K),t/z,a,c<1km,d<1km,f<1km,c>1km,d>1km,f>1km
0,2016-11-01,STA-BG0052A,692.88,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0
1,2016-11-01,STA-BG0050A,823.44,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0
2,2016-11-01,STA-BG0073A,624.00,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0
3,2016-11-01,STA-BG0040A,876.24,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0
4,2016-11-02,STA-BG0052A,1632.96,E,1503,10.0,283.0,0.002257,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0


In [56]:
#new feature day of week
df_stations_merged['Date'] = pd.to_datetime(df_stations_merged['Date'])
df_stations_merged['day_of_week'] = df_stations_merged['Date'].dt.day_name()
df_stations_merged.head()

,Date,Station,PM10,PCS,HGHT(m),TEMP(C),TEMP(K),t/z,a,c<1km,d<1km,f<1km,c>1km,d>1km,f>1km,day_of_week
0,2016-11-01,STA-BG0052A,692.88,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0,Tuesday
1,2016-11-01,STA-BG0050A,823.44,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0,Tuesday
2,2016-11-01,STA-BG0073A,624.00,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0,Tuesday
3,2016-11-01,STA-BG0040A,876.24,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0,Tuesday
4,2016-11-02,STA-BG0052A,1632.96,E,1503,10.0,283.0,0.002257,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0,Wednesday


### Industrial distances

In [57]:
df_industrial.head()

,X*,Y*,m,t/y
0,"'""42°44''16.66""""N""'","'""23°14''28.82""""E""'",8.0,0.38
1,"'""42°39''46.01""""N""'","'""23°23''19.70""""E""'",15.0,0.03
2,"'""42°39''46.47""""N""'","'""23°23''19.27""""E""'",15.0,0.20
3,"'""42°39''46.70""""N""'","'""23°23''19.07""""E""'",15.0,0.96
4,"'""42°39''47.12""""N""'","'""23°23''21.30""""E""'",15.0,1.58


In [58]:
from math import cos, sqrt
fixedIndustrialDf = pd.read_csv("Data/industrial_coord.txt")

def get_distance(Lat1, Long1, Lat2, Long2):
    x = Lat2 - Lat1
    y = (Long2 - Long1)*cos((Lat2 + Lat1)*0.00872664626)  
    return 111.138*sqrt(x*x+y*y)



pd.options.mode.chained_assignment = None  # default='warn'

factoryDistanceDf = pd.DataFrame([['STA-BG0040A'], ['STA-BG0050A'], ['STA-BG0052A'], ['STA-BG0073A'] ], columns=['Name'])

for i in range(df_industrial.shape[0]):
    #columnName = 'dist' + str(i)
    #factoryDistanceDf[columnName] = ''
    xCol = 'x' + str(i)
    factoryDistanceDf[xCol] = ''
    yCol = 'y' + str(i)
    factoryDistanceDf[yCol] = ''
    
    
    for j in range(factoryDistanceDf.shape[0]):
        stationLat = df_stations_loc.iloc[j]['Latitude']
        stationLon = df_stations_loc.iloc[j]['Longitude']
        
        factoryLat = fixedIndustrialDf.iloc[i]['Lat']
        factoryLon = fixedIndustrialDf.iloc[i]['Lon']
        
        #distance = get_distance(stationLat, stationLon, factoryLat, factoryLon)
        
        x = get_distance(stationLat, 0, factoryLat, 0)
        y = get_distance(0, stationLon, 0, factoryLon)

        #print(distance)
        #factoryDistanceDf.loc[j][columnName] = distance
        factoryDistanceDf.loc[j][xCol] = x
        factoryDistanceDf.loc[j][yCol] = y

In [59]:
# calculated distances
factoryDistanceDf.head()

,Name,x0,y0,x1,y1,x2,y2,x3,y3,x4,...,x66,y66,x67,y67,x68,y68,x69,y69,x70,y70
0,STA-BG0040A,0.630054,7.73888,7.72536,8.65027,7.71116,8.63699,7.70406,8.63082,7.6911,...,1.91493,6.40492,1.88869,6.42221,1.90258,6.4253,3.56842,14.0899,3.56224,14.0979
1,STA-BG0050A,6.37967,6.16228,1.97575,10.2269,1.96155,10.2136,1.95445,10.2074,1.94148,...,7.66455,4.82832,7.6383,4.8456,7.6522,4.84869,9.31803,15.6665,9.31186,15.6745
2,STA-BG0052A,7.94116,17.6515,0.414261,1.26235,0.40006,1.27563,0.392959,1.2818,0.379993,...,9.22603,16.3175,9.19979,16.3348,9.21369,16.3379,10.8795,4.17723,10.8733,4.18526
3,STA-BG0073A,7.57562,3.00785,0.779794,13.3813,0.765593,13.368,0.758492,13.3619,0.745526,...,8.8605,1.67389,8.83426,1.69117,8.84815,1.69426,10.514,18.8209,10.5078,18.8289


In [60]:
df_ind_model = pd.merge(df_stations_merged,factoryDistanceDf, how='left', left_on='Station', right_on='Name')
df_ind_model.columns

Index(['Date', 'Station', 'PM10', 'PCS', 'HGHT(m)', 'TEMP(C)', 'TEMP(K)',
       't/z', 'a', 'c<1km',
       ...
       'x66', 'y66', 'x67', 'y67', 'x68', 'y68', 'x69', 'y69', 'x70', 'y70'],
      dtype='object', length=159)

In [61]:
def calculateSigmaY(x, a):
    #sigmaY = a*x^0.894
    sigmaY = a * pow(x, 0.894)
    return sigmaY
 
def calculateSigmaZ(x, c, d, f):
    #sigmaZ = cx^(d+f)
    sigmaZ = c * pow(x, d)+f
    return sigmaZ
 
def calculateSigmaZFromRow(row, index):
    sigmaZ = calculateSigmaZ(row['x' + str(index)], row['c>1km'], row['d>1km'], row['f>1km'])
    return sigmaZ
 
def calculateSigmaYFromRow(row, index):
    sigmaY = calculateSigmaY(row['y' + str(index)], row['a'])
    return sigmaY
 
for index in range(0, df_industrial.shape[0]):
    df_ind_model['sigmaZ' + str(index)] = df_ind_model.apply(lambda row: calculateSigmaZFromRow(row, index), axis=1)
    df_ind_model['sigmaY' + str(index)] = df_ind_model.apply(lambda row: calculateSigmaYFromRow(row, index), axis=1)

In [62]:
#Old 
#df_ind_model.head()

In [63]:
for column in df_ind_model:
    print(column)

Date
Station
PM10
PCS
HGHT(m)
TEMP(C)
TEMP(K)
t/z
a
c<1km
d<1km
f<1km
c>1km
d>1km
f>1km
day_of_week
Name
x0
y0
x1
y1
x2
y2
x3
y3
x4
y4
x5
y5
x6
y6
x7
y7
x8
y8
x9
y9
x10
y10
x11
y11
x12
y12
x13
y13
x14
y14
x15
y15
x16
y16
x17
y17
x18
y18
x19
y19
x20
y20
x21
y21
x22
y22
x23
y23
x24
y24
x25
y25
x26
y26
x27
y27
x28
y28
x29
y29
x30
y30
x31
y31
x32
y32
x33
y33
x34
y34
x35
y35
x36
y36
x37
y37
x38
y38
x39
y39
x40
y40
x41
y41
x42
y42
x43
y43
x44
y44
x45
y45
x46
y46
x47
y47
x48
y48
x49
y49
x50
y50
x51
y51
x52
y52
x53
y53
x54
y54
x55
y55
x56
y56
x57
y57
x58
y58
x59
y59
x60
y60
x61
y61
x62
y62
x63
y63
x64
y64
x65
y65
x66
y66
x67
y67
x68
y68
x69
y69
x70
y70
sigmaZ0
sigmaY0
sigmaZ1
sigmaY1
sigmaZ2
sigmaY2
sigmaZ3
sigmaY3
sigmaZ4
sigmaY4
sigmaZ5
sigmaY5
sigmaZ6
sigmaY6
sigmaZ7
sigmaY7
sigmaZ8
sigmaY8
sigmaZ9
sigmaY9
sigmaZ10
sigmaY10
sigmaZ11
sigmaY11
sigmaZ12
sigmaY12
sigmaZ13
sigmaY13
sigmaZ14
sigmaY14
sigmaZ15
sigmaY15
sigmaZ16
sigmaY16
sigmaZ17
sigmaY17
sigmaZ18
sigmaY18
sigmaZ19
sigmaY19
sigmaZ2

In [64]:
df_ind_model.columns

Index(['Date', 'Station', 'PM10', 'PCS', 'HGHT(m)', 'TEMP(C)', 'TEMP(K)',
       't/z', 'a', 'c<1km',
       ...
       'sigmaZ66', 'sigmaY66', 'sigmaZ67', 'sigmaY67', 'sigmaZ68', 'sigmaY68',
       'sigmaZ69', 'sigmaY69', 'sigmaZ70', 'sigmaY70'],
      dtype='object', length=301)

In [65]:
df_ind_model['PM10micro'] = df_ind_model['PM10'] / 1.225

In [66]:
# for indexheight in range(0,71):
#     df_ind_model['m'+str(indexheight)] = df_industrial.loc['m']

### Weather

In [67]:
df_weather.head()

,year,Month,day,TASMAX,TASAVG,TASMIN,DPMAX,DPAVG,DPMIN,RHMAX,...,sfcWindMAX,sfcWindAVG,sfcWindMIN,PSLMAX,PSLAVG,PSLMIN,PRCPMAX,PRCPAVG,PRCPMIN,VISIB
0,2016,11,1,12.7788,6.6672,0.0000,2.2224,-1.1112,-3.3336,87,...,24.14010,12.07005,0.0,1026.753044,1024.890530,1023.028016,-9999,0.00,-9999,5.471756
1,2016,11,2,15.5568,6.6672,-1.6668,2.7780,0.5556,-2.7780,100,...,11.26538,5.63269,0.0,1026.414405,1020.826864,1015.239322,-9999,0.00,-9999,8.046700
2,2016,11,3,13.3344,8.3340,3.3336,7.2228,3.3336,-1.1112,100,...,28.96812,14.48406,0.0,1023.366655,1019.133669,1014.900683,-9999,5.08,-9999,7.563898
3,2016,11,4,10.5564,6.1116,1.6668,6.1116,3.3336,1.1112,100,...,28.96812,14.48406,0.0,1025.398488,1023.197336,1020.996183,-9999,0.00,-9999,9.816974
4,2016,11,5,15.0012,8.8896,2.7780,10.0008,6.6672,2.7780,100,...,14.48406,7.24203,0.0,1024.043933,1020.657544,1017.271155,-9999,0.00,-9999,9.977908


In [131]:
def formatDateOfMonth(date):
    if len(str(date)) == 1:
        return '0'+ str(date)
    return date

df = df_weather
df['year'] = df['year'].astype(str)
df['Month'] = df['Month'].astype(str)
df['day'] = df['day'].apply(formatDateOfMonth)
df['day'] = df['day'].astype(str)

df['Date'] = df['year'].str.cat(df['Month'], sep='-')
df['Date'] = df['Date'].str.cat(df['day'], sep='-')#  +  + '-' + df['day']
df_weather = df

In [69]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 23 columns):
year          20 non-null object
Month         20 non-null object
day           20 non-null object
TASMAX        20 non-null float64
TASAVG        20 non-null float64
TASMIN        20 non-null float64
DPMAX         20 non-null float64
DPAVG         20 non-null float64
DPMIN         20 non-null float64
RHMAX         20 non-null int64
RHAVG         20 non-null float64
RHMIN         20 non-null int64
sfcWindMAX    20 non-null float64
sfcWindAVG    20 non-null float64
sfcWindMIN    20 non-null float64
PSLMAX        20 non-null float64
PSLAVG        20 non-null float64
PSLMIN        20 non-null float64
PRCPMAX       20 non-null int64
PRCPAVG       20 non-null float64
PRCPMIN       20 non-null int64
VISIB         20 non-null float64
Date          20 non-null object
dtypes: float64(15), int64(4), object(4)
memory usage: 3.7+ KB


In [70]:
df_industrial.head()

,X*,Y*,m,t/y
0,"'""42°44''16.66""""N""'","'""23°14''28.82""""E""'",8.0,0.38
1,"'""42°39''46.01""""N""'","'""23°23''19.70""""E""'",15.0,0.03
2,"'""42°39''46.47""""N""'","'""23°23''19.27""""E""'",15.0,0.20
3,"'""42°39''46.70""""N""'","'""23°23''19.07""""E""'",15.0,0.96
4,"'""42°39''47.12""""N""'","'""23°23''21.30""""E""'",15.0,1.58


### Final Merge

In [71]:
df_ind_model['Date'] = df_ind_model['Date'].astype(str)
df_ind_model['Date'] = df_ind_model['Date'].astype(str)

In [72]:
df_ind_model = pd.merge(df_ind_model,df_weather, how='left', left_on='Date', right_on='Date')
df_ind_model

,Date,Station,PM10,PCS,HGHT(m),TEMP(C),TEMP(K),t/z,a,c<1km,...,sfcWindMAX,sfcWindAVG,sfcWindMIN,PSLMAX,PSLAVG,PSLMIN,PRCPMAX,PRCPAVG,PRCPMIN,VISIB
0,2016-11-01,STA-BG0052A,692.880,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,24.14010,12.07005,0.00000,1026.753044,1024.890530,1023.028016,-9999,0.000,-9999,5.471756
1,2016-11-01,STA-BG0050A,823.440,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,24.14010,12.07005,0.00000,1026.753044,1024.890530,1023.028016,-9999,0.000,-9999,5.471756
2,2016-11-01,STA-BG0073A,624.000,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,24.14010,12.07005,0.00000,1026.753044,1024.890530,1023.028016,-9999,0.000,-9999,5.471756
3,2016-11-01,STA-BG0040A,876.240,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,24.14010,12.07005,0.00000,1026.753044,1024.890530,1023.028016,-9999,0.000,-9999,5.471756
4,2016-11-02,STA-BG0052A,1632.960,E,1503,10.0,283.0,0.002257,50.5,22.8,...,11.26538,5.63269,0.00000,1026.414405,1020.826864,1015.239322,-9999,0.000,-9999,8.046700
5,2016-11-02,STA-BG0050A,1756.560,E,1503,10.0,283.0,0.002257,50.5,22.8,...,11.26538,5.63269,0.00000,1026.414405,1020.826864,1015.239322,-9999,0.000,-9999,8.046700
6,2016-11-02,STA-BG0073A,1516.560,E,1503,10.0,283.0,0.002257,50.5,22.8,...,11.26538,5.63269,0.00000,1026.414405,1020.826864,1015.239322,-9999,0.000,-9999,8.046700
7,2016-11-02,STA-BG0040A,2382.288,E,1503,10.0,283.0,0.002257,50.5,22.8,...,11.26538,5.63269,0.00000,1026.414405,1020.826864,1015.239322,-9999,0.000,-9999,8.046700
8,2016-11-03,STA-BG0052A,953.280,E,1487,13.2,280.0,0.003618,50.5,22.8,...,28.96812,14.48406,0.00000,1023.366655,1019.133669,1014.900683,-9999,5.080,-9999,7.563898
9,2016-11-03,STA-BG0050A,978.480,E,1487,13.2,280.0,0.003618,50.5,22.8,...,28.96812,14.48406,0.00000,1023.366655,1019.133669,1014.900683,-9999,5.080,-9999,7.563898


In [73]:
list(df_ind_model.columns)

['Date',
 'Station',
 'PM10',
 'PCS',
 'HGHT(m)',
 'TEMP(C)',
 'TEMP(K)',
 't/z',
 'a',
 'c<1km',
 'd<1km',
 'f<1km',
 'c>1km',
 'd>1km',
 'f>1km',
 'day_of_week',
 'Name',
 'x0',
 'y0',
 'x1',
 'y1',
 'x2',
 'y2',
 'x3',
 'y3',
 'x4',
 'y4',
 'x5',
 'y5',
 'x6',
 'y6',
 'x7',
 'y7',
 'x8',
 'y8',
 'x9',
 'y9',
 'x10',
 'y10',
 'x11',
 'y11',
 'x12',
 'y12',
 'x13',
 'y13',
 'x14',
 'y14',
 'x15',
 'y15',
 'x16',
 'y16',
 'x17',
 'y17',
 'x18',
 'y18',
 'x19',
 'y19',
 'x20',
 'y20',
 'x21',
 'y21',
 'x22',
 'y22',
 'x23',
 'y23',
 'x24',
 'y24',
 'x25',
 'y25',
 'x26',
 'y26',
 'x27',
 'y27',
 'x28',
 'y28',
 'x29',
 'y29',
 'x30',
 'y30',
 'x31',
 'y31',
 'x32',
 'y32',
 'x33',
 'y33',
 'x34',
 'y34',
 'x35',
 'y35',
 'x36',
 'y36',
 'x37',
 'y37',
 'x38',
 'y38',
 'x39',
 'y39',
 'x40',
 'y40',
 'x41',
 'y41',
 'x42',
 'y42',
 'x43',
 'y43',
 'x44',
 'y44',
 'x45',
 'y45',
 'x46',
 'y46',
 'x47',
 'y47',
 'x48',
 'y48',
 'x49',
 'y49',
 'x50',
 'y50',
 'x51',
 'y51',
 'x52',
 'y52',

## Define and calculate C(x,y,z)

In [74]:
def C_xyz(Q,u,sigmaY,sigmaZ,h,z,y):
#    Cxyz = ((np.e**((-(z-h)**2)/(2*sigmaZ**2)))+(np.e**((-(z+h)**2)/(2*sigmaZ**2))))
    Cxyz = (Q/(2*np.pi*u*sigmaY*sigmaZ))*(np.e**((y**2)/(2*sigmaY**2)))*((np.e**((-(z-h)**2)/(2*sigmaZ**2)))+(np.e**((-(z+h)**2)/(2*sigmaZ**2))))
    return Cxyz

In [75]:
# list(df_ind_model.columns)

#### Assumptions:
- y=x becuase the pollutions spreads radially
- z = 0, becuase of the ground level
- h is the height of the chimney

In [76]:
def calculateCxyz(row, index):
    Cxyz = C_xyz(Q=row['PM10micro'],
                 u=row['sfcWindAVG'],
#                  sigmaY = 300,
                 sigmaY = row['sigmaY' + str(index)], 
#                  sigmaZ = 300,
                 sigmaZ = row['sigmaZ' + str(index)],
                 h = df_industrial.loc[index]['m'],
                 z = 0, 
                 y = row['y' + str(index)])
    return Cxyz

df_ind_model['Ctotal'] = 0
 
for index in range(0, df_industrial.shape[0]):
    df_ind_model['C' + str(index)] = df_ind_model.apply(lambda row: calculateCxyz(row, index), axis=1)
    df_ind_model['Ctotal'] = df_ind_model['Ctotal'] + df_ind_model['C' + str(index)]

In [77]:
df_ind_model

,Date,Station,PM10,PCS,HGHT(m),TEMP(C),TEMP(K),t/z,a,c<1km,...,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70
0,2016-11-01,STA-BG0052A,692.880,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,1.408322e-06,0.000837,6.001781e-04,0.000687,0.000678,9.044566e-05,0.000318,0.000318,0.001008,0.001017
1,2016-11-01,STA-BG0050A,823.440,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,1.673836e-05,0.000633,1.624575e-03,0.001884,0.001878,2.753297e-04,0.001212,0.001211,0.000393,0.000397
2,2016-11-01,STA-BG0073A,624.000,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,9.866119e-07,0.000337,6.499269e-04,0.000738,0.000734,6.060122e-04,0.002212,0.002207,0.000240,0.000242
3,2016-11-01,STA-BG0040A,876.240,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,1.338528e-04,0.001019,-2.335282e-172,-0.000021,-0.000010,3.518470e-06,0.001925,0.001917,0.000714,0.000734
4,2016-11-02,STA-BG0052A,1632.960,E,1503,10.0,283.0,0.002257,50.5,22.8,...,7.112345e-06,0.004227,3.031035e-03,0.003471,0.003423,4.567710e-04,0.001605,0.001604,0.005092,0.005135
5,2016-11-02,STA-BG0050A,1756.560,E,1503,10.0,283.0,0.002257,50.5,22.8,...,7.651334e-05,0.002891,7.426157e-03,0.008610,0.008582,1.258570e-03,0.005542,0.005535,0.001795,0.001815
6,2016-11-02,STA-BG0073A,1516.560,E,1503,10.0,283.0,0.002257,50.5,22.8,...,5.138242e-06,0.001755,3.384798e-03,0.003846,0.003825,3.156092e-03,0.011518,0.011491,0.001249,0.001262
7,2016-11-02,STA-BG0040A,2382.288,E,1503,10.0,283.0,0.002257,50.5,22.8,...,7.798158e-04,0.005938,-1.360516e-171,-0.000122,-0.000056,2.049832e-05,0.011212,0.011167,0.004157,0.004278
8,2016-11-03,STA-BG0052A,953.280,E,1487,13.2,280.0,0.003618,50.5,22.8,...,1.614668e-06,0.000960,6.881155e-04,0.000788,0.000777,1.036977e-04,0.000364,0.000364,0.001156,0.001166
9,2016-11-03,STA-BG0050A,978.480,E,1487,13.2,280.0,0.003618,50.5,22.8,...,1.657493e-05,0.000626,1.608713e-03,0.001865,0.001859,2.726414e-04,0.001201,0.001199,0.000389,0.000393


In [78]:
df_ind_model.Ctotal.sum()

15.902878906305947

# Level 2

The calculation of the stability class gives E class for all dates. We automatically pick the most frequent class in case it is different.

### Stations

In [79]:
df_stations_merged.head()

,Date,Station,PM10,PCS,HGHT(m),TEMP(C),TEMP(K),t/z,a,c<1km,d<1km,f<1km,c>1km,d>1km,f>1km,day_of_week
0,2016-11-01,STA-BG0052A,692.88,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0,Tuesday
1,2016-11-01,STA-BG0050A,823.44,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0,Tuesday
2,2016-11-01,STA-BG0073A,624.00,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0,Tuesday
3,2016-11-01,STA-BG0040A,876.24,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0,Tuesday
4,2016-11-02,STA-BG0052A,1632.96,E,1503,10.0,283.0,0.002257,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0,Wednesday


In [80]:
#new feature day of week
df_stations_merged.head()

,Date,Station,PM10,PCS,HGHT(m),TEMP(C),TEMP(K),t/z,a,c<1km,d<1km,f<1km,c>1km,d>1km,f>1km,day_of_week
0,2016-11-01,STA-BG0052A,692.88,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0,Tuesday
1,2016-11-01,STA-BG0050A,823.44,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0,Tuesday
2,2016-11-01,STA-BG0073A,624.00,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0,Tuesday
3,2016-11-01,STA-BG0040A,876.24,E,1823,-1.4,276.6,0.003299,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0,Tuesday
4,2016-11-02,STA-BG0052A,1632.96,E,1503,10.0,283.0,0.002257,50.5,22.8,0.678,-1.3,55.4,0.305,-34.0,Wednesday


### Industrial distances

In [110]:
df_household.head()

,X,Y,NJ16_eq_1,NJ16_eq_2,NJ16_eq_3,NJ17_eq_1,NJ17_eq_3,NJ17_eq_4,NJ17_eq_6,NJ17_eq_7,NJ17_eq_4i,NJ17_eq_8,NJ17_eq_9,NN_Jilisht,NBROI_LICA
0,23.383978,42.693180,NaN,NaN,1.0,NaN,NaN,0,1,0,1,NaN,NaN,1,4
1,23.383978,42.693180,NaN,NaN,1.0,NaN,NaN,0,1,0,1,NaN,NaN,1,2
2,23.382120,42.693912,NaN,1.0,NaN,NaN,NaN,0,1,0,1,NaN,NaN,1,4
3,23.376602,42.678282,NaN,NaN,1.0,NaN,NaN,0,0,1,1,NaN,NaN,1,4
4,23.383978,42.693180,NaN,NaN,1.0,NaN,NaN,0,1,0,1,NaN,NaN,1,1


In [111]:
df_household.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39944 entries, 0 to 39943
Data columns (total 15 columns):
X             39944 non-null float64
Y             39944 non-null float64
NJ16_eq_1     3899 non-null float64
NJ16_eq_2     24912 non-null float64
NJ16_eq_3     18879 non-null float64
NJ17_eq_1     1300 non-null float64
NJ17_eq_3     5200 non-null float64
NJ17_eq_4     39944 non-null int64
NJ17_eq_6     39944 non-null int64
NJ17_eq_7     39944 non-null int64
NJ17_eq_4i    39944 non-null int64
NJ17_eq_8     423 non-null float64
NJ17_eq_9     3652 non-null float64
NN_Jilisht    39944 non-null int64
NBROI_LICA    39944 non-null int64
dtypes: float64(9), int64(6)
memory usage: 4.6 MB


In [112]:
df_stations_loc

,AirQualityStationEoICode,CommonName,Longitude,Latitude
0,BG0040A,Nadezhda,23.310972,42.732292
1,BG0050A,Hipodruma,23.296786,42.680558
2,BG0052A,Druzhba,23.400164,42.666508
3,BG0073A,IAOS/Pavlovo,23.268403,42.669797


In [117]:
from math import cos, sqrt
fixedhouseholdDf = df_household

def get_distance(Lat1, Long1, Lat2, Long2):
    x = Lat2 - Lat1
    y = (Long2 - Long1)*cos((Lat2 + Lat1)*0.00872664626)  
    return 111.138*sqrt(x*x+y*y)



pd.options.mode.chained_assignment = None  # default='warn'

householdDistanceDf = pd.DataFrame([['STA-BG0040A'], ['STA-BG0050A'], ['STA-BG0052A'], ['STA-BG0073A'] ], columns=['Name'])

# for i in range(df_household.shape[0]):
for i in range(1000):
    #columnName = 'dist' + str(i)
    #factoryDistanceDf[columnName] = ''
    xCol = 'x' + str(i)
    householdDistanceDf[xCol] = ''
    yCol = 'y' + str(i)
    householdDistanceDf[yCol] = ''
    
    
    for j in range(factoryDistanceDf.shape[0]):
        stationLat = df_stations_loc.iloc[j]['Latitude']
        stationLon = df_stations_loc.iloc[j]['Longitude']
        
        hhLat = fixedhouseholdDf.iloc[i]['Y']
        hhLon = fixedhouseholdDf.iloc[i]['X']
        
        #distance = get_distance(stationLat, stationLon, factoryLat, factoryLon)
        
        x = get_distance(stationLat, 0, hhLat, 0)
        y = get_distance(0, stationLon, 0, hhLon)

        #print(distance)
        #factoryDistanceDf.loc[j][columnName] = distance
        householdDistanceDf.loc[j][xCol] = x
        householdDistanceDf.loc[j][yCol] = y

In [118]:
# calculated distances
householdDistanceDf.head()

,Name,x0,y0,x1,y1,x2,y2,x3,y3,x4,...,x995,y995,x996,y996,x997,y997,x998,y998,x999,y999
0,STA-BG0040A,4.34681,8.11376,4.34681,8.11376,4.26549,7.90719,6.00253,7.29402,4.34681,...,1.0845,3.39081,1.09909,3.40361,1.09909,3.40361,6.66448,8.36691,1.0845,3.39081
1,STA-BG0050A,1.40281,9.69037,1.40281,9.69037,1.48413,9.48379,0.252917,8.87063,1.40281,...,4.66512,4.96741,4.65053,4.98022,4.65053,4.98022,0.914865,6.79031,4.66512,4.96741
2,STA-BG0052A,2.96429,1.79886,2.96429,1.79886,3.04561,2.00543,1.30857,2.6186,2.96429,...,6.22661,6.52181,6.21201,6.50901,6.21201,6.50901,0.646624,18.2795,6.22661,6.52181
3,STA-BG0073A,2.59876,12.8448,2.59876,12.8448,2.68008,12.6382,0.943039,12.0251,2.59876,...,5.86107,8.12184,5.84648,8.13465,5.84648,8.13465,0.281091,3.63588,5.86107,8.12184


In [124]:
df_hh_model = pd.merge(df_stations_merged,householdDistanceDf, how='left', left_on='Station', right_on='Name')
df_hh_model.head()

,Date,Station,PM10,PCS,HGHT(m),TEMP(C),TEMP(K),t/z,a,c<1km,...,x995,y995,x996,y996,x997,y997,x998,y998,x999,y999
0,2016-11-01,STA-BG0052A,692.88,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,6.22661,6.52181,6.21201,6.50901,6.21201,6.50901,0.646624,18.2795,6.22661,6.52181
1,2016-11-01,STA-BG0050A,823.44,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,4.66512,4.96741,4.65053,4.98022,4.65053,4.98022,0.914865,6.79031,4.66512,4.96741
2,2016-11-01,STA-BG0073A,624.00,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,5.86107,8.12184,5.84648,8.13465,5.84648,8.13465,0.281091,3.63588,5.86107,8.12184
3,2016-11-01,STA-BG0040A,876.24,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,1.0845,3.39081,1.09909,3.40361,1.09909,3.40361,6.66448,8.36691,1.0845,3.39081
4,2016-11-02,STA-BG0052A,1632.96,E,1503,10.0,283.0,0.002257,50.5,22.8,...,6.22661,6.52181,6.21201,6.50901,6.21201,6.50901,0.646624,18.2795,6.22661,6.52181


In [120]:
def calculateSigmaY(x, a):
    #sigmaY = a*x^0.894
    sigmaY = a * pow(x, 0.894)
    return sigmaY
 
def calculateSigmaZ(x, c, d, f):
    #sigmaZ = cx^(d+f)
    sigmaZ = c * pow(x, d)+f
    return sigmaZ
 
def calculateSigmaZFromRow(row, index):
    sigmaZ = calculateSigmaZ(row['x' + str(index)], row['c>1km'], row['d>1km'], row['f>1km'])
    return sigmaZ
 
def calculateSigmaYFromRow(row, index):
    sigmaY = calculateSigmaY(row['y' + str(index)], row['a'])
    return sigmaY
 
for index in range(0, df_industrial.shape[0]):
    df_hh_model['sigmaZ' + str(index)] = df_hh_model.apply(lambda row: calculateSigmaZFromRow(row, index), axis=1)
    df_hh_model['sigmaY' + str(index)] = df_hh_model.apply(lambda row: calculateSigmaYFromRow(row, index), axis=1)

In [62]:
#Old 
#df_ind_model.head()

In [63]:
for column in df_ind_model:
    print(column)

Date
Station
PM10
PCS
HGHT(m)
TEMP(C)
TEMP(K)
t/z
a
c<1km
d<1km
f<1km
c>1km
d>1km
f>1km
day_of_week
Name
x0
y0
x1
y1
x2
y2
x3
y3
x4
y4
x5
y5
x6
y6
x7
y7
x8
y8
x9
y9
x10
y10
x11
y11
x12
y12
x13
y13
x14
y14
x15
y15
x16
y16
x17
y17
x18
y18
x19
y19
x20
y20
x21
y21
x22
y22
x23
y23
x24
y24
x25
y25
x26
y26
x27
y27
x28
y28
x29
y29
x30
y30
x31
y31
x32
y32
x33
y33
x34
y34
x35
y35
x36
y36
x37
y37
x38
y38
x39
y39
x40
y40
x41
y41
x42
y42
x43
y43
x44
y44
x45
y45
x46
y46
x47
y47
x48
y48
x49
y49
x50
y50
x51
y51
x52
y52
x53
y53
x54
y54
x55
y55
x56
y56
x57
y57
x58
y58
x59
y59
x60
y60
x61
y61
x62
y62
x63
y63
x64
y64
x65
y65
x66
y66
x67
y67
x68
y68
x69
y69
x70
y70
sigmaZ0
sigmaY0
sigmaZ1
sigmaY1
sigmaZ2
sigmaY2
sigmaZ3
sigmaY3
sigmaZ4
sigmaY4
sigmaZ5
sigmaY5
sigmaZ6
sigmaY6
sigmaZ7
sigmaY7
sigmaZ8
sigmaY8
sigmaZ9
sigmaY9
sigmaZ10
sigmaY10
sigmaZ11
sigmaY11
sigmaZ12
sigmaY12
sigmaZ13
sigmaY13
sigmaZ14
sigmaY14
sigmaZ15
sigmaY15
sigmaZ16
sigmaY16
sigmaZ17
sigmaY17
sigmaZ18
sigmaY18
sigmaZ19
sigmaY19
sigmaZ2

In [143]:
df_hh_model['PM10micro'] = df_hh_model['PM10'] / 1.225

In [145]:
df_hh_model.PM10.head()

0     692.88
1     823.44
2     624.00
3     876.24
4    1632.96
Name: PM10, dtype: float64

In [66]:
# for indexheight in range(0,71):
#     df_ind_model['m'+str(indexheight)] = df_industrial.loc['m']

### Final Merge

In [137]:
df_hh_model['Date'] = df_hh_model['Date'].astype(str)
df_weather['Date'] = df_weather['Date'].astype(str)

In [138]:
df_weather.head()

,year,Month,day,TASMAX,TASAVG,TASMIN,DPMAX,DPAVG,DPMIN,RHMAX,...,sfcWindAVG,sfcWindMIN,PSLMAX,PSLAVG,PSLMIN,PRCPMAX,PRCPAVG,PRCPMIN,VISIB,Date
0,2016,11,01,12.7788,6.6672,0.0000,2.2224,-1.1112,-3.3336,87,...,12.07005,0.0,1026.753044,1024.890530,1023.028016,-9999,0.00,-9999,5.471756,2016-11-01
1,2016,11,02,15.5568,6.6672,-1.6668,2.7780,0.5556,-2.7780,100,...,5.63269,0.0,1026.414405,1020.826864,1015.239322,-9999,0.00,-9999,8.046700,2016-11-02
2,2016,11,03,13.3344,8.3340,3.3336,7.2228,3.3336,-1.1112,100,...,14.48406,0.0,1023.366655,1019.133669,1014.900683,-9999,5.08,-9999,7.563898,2016-11-03
3,2016,11,04,10.5564,6.1116,1.6668,6.1116,3.3336,1.1112,100,...,14.48406,0.0,1025.398488,1023.197336,1020.996183,-9999,0.00,-9999,9.816974,2016-11-04
4,2016,11,05,15.0012,8.8896,2.7780,10.0008,6.6672,2.7780,100,...,7.24203,0.0,1024.043933,1020.657544,1017.271155,-9999,0.00,-9999,9.977908,2016-11-05


In [139]:
df_hh_model = pd.merge(df_hh_model,df_weather, how='left', left_on='Date', right_on='Date')
df_hh_model

,Date,Station,PM10,PCS,HGHT(m),TEMP(C),TEMP(K),t/z,a,c<1km,...,sfcWindMAX,sfcWindAVG,sfcWindMIN,PSLMAX,PSLAVG,PSLMIN,PRCPMAX,PRCPAVG,PRCPMIN,VISIB
0,2016-11-01,STA-BG0052A,692.880,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,24.14010,12.07005,0.00000,1026.753044,1024.890530,1023.028016,-9999,0.000,-9999,5.471756
1,2016-11-01,STA-BG0050A,823.440,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,24.14010,12.07005,0.00000,1026.753044,1024.890530,1023.028016,-9999,0.000,-9999,5.471756
2,2016-11-01,STA-BG0073A,624.000,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,24.14010,12.07005,0.00000,1026.753044,1024.890530,1023.028016,-9999,0.000,-9999,5.471756
3,2016-11-01,STA-BG0040A,876.240,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,24.14010,12.07005,0.00000,1026.753044,1024.890530,1023.028016,-9999,0.000,-9999,5.471756
4,2016-11-02,STA-BG0052A,1632.960,E,1503,10.0,283.0,0.002257,50.5,22.8,...,11.26538,5.63269,0.00000,1026.414405,1020.826864,1015.239322,-9999,0.000,-9999,8.046700
5,2016-11-02,STA-BG0050A,1756.560,E,1503,10.0,283.0,0.002257,50.5,22.8,...,11.26538,5.63269,0.00000,1026.414405,1020.826864,1015.239322,-9999,0.000,-9999,8.046700
6,2016-11-02,STA-BG0073A,1516.560,E,1503,10.0,283.0,0.002257,50.5,22.8,...,11.26538,5.63269,0.00000,1026.414405,1020.826864,1015.239322,-9999,0.000,-9999,8.046700
7,2016-11-02,STA-BG0040A,2382.288,E,1503,10.0,283.0,0.002257,50.5,22.8,...,11.26538,5.63269,0.00000,1026.414405,1020.826864,1015.239322,-9999,0.000,-9999,8.046700
8,2016-11-03,STA-BG0052A,953.280,E,1487,13.2,280.0,0.003618,50.5,22.8,...,28.96812,14.48406,0.00000,1023.366655,1019.133669,1014.900683,-9999,5.080,-9999,7.563898
9,2016-11-03,STA-BG0050A,978.480,E,1487,13.2,280.0,0.003618,50.5,22.8,...,28.96812,14.48406,0.00000,1023.366655,1019.133669,1014.900683,-9999,5.080,-9999,7.563898


In [140]:
list(df_ind_model.columns)

['Date',
 'Station',
 'PM10',
 'PCS',
 'HGHT(m)',
 'TEMP(C)',
 'TEMP(K)',
 't/z',
 'a',
 'c<1km',
 'd<1km',
 'f<1km',
 'c>1km',
 'd>1km',
 'f>1km',
 'day_of_week',
 'Name',
 'x0',
 'y0',
 'x1',
 'y1',
 'x2',
 'y2',
 'x3',
 'y3',
 'x4',
 'y4',
 'x5',
 'y5',
 'x6',
 'y6',
 'x7',
 'y7',
 'x8',
 'y8',
 'x9',
 'y9',
 'x10',
 'y10',
 'x11',
 'y11',
 'x12',
 'y12',
 'x13',
 'y13',
 'x14',
 'y14',
 'x15',
 'y15',
 'x16',
 'y16',
 'x17',
 'y17',
 'x18',
 'y18',
 'x19',
 'y19',
 'x20',
 'y20',
 'x21',
 'y21',
 'x22',
 'y22',
 'x23',
 'y23',
 'x24',
 'y24',
 'x25',
 'y25',
 'x26',
 'y26',
 'x27',
 'y27',
 'x28',
 'y28',
 'x29',
 'y29',
 'x30',
 'y30',
 'x31',
 'y31',
 'x32',
 'y32',
 'x33',
 'y33',
 'x34',
 'y34',
 'x35',
 'y35',
 'x36',
 'y36',
 'x37',
 'y37',
 'x38',
 'y38',
 'x39',
 'y39',
 'x40',
 'y40',
 'x41',
 'y41',
 'x42',
 'y42',
 'x43',
 'y43',
 'x44',
 'y44',
 'x45',
 'y45',
 'x46',
 'y46',
 'x47',
 'y47',
 'x48',
 'y48',
 'x49',
 'y49',
 'x50',
 'y50',
 'x51',
 'y51',
 'x52',
 'y52',

## Define and calculate C(x,y,z)

In [141]:
def C_xyz(Q,u,sigmaY,sigmaZ,h,z,y):
#    Cxyz = ((np.e**((-(z-h)**2)/(2*sigmaZ**2)))+(np.e**((-(z+h)**2)/(2*sigmaZ**2))))
    Cxyz = (Q/(2*np.pi*u*sigmaY*sigmaZ))*(np.e**((y**2)/(2*sigmaY**2)))*((np.e**((-(z-h)**2)/(2*sigmaZ**2)))+(np.e**((-(z+h)**2)/(2*sigmaZ**2))))
    return Cxyz

In [75]:
# list(df_ind_model.columns)

#### Assumptions:
- y=x becuase the pollutions spreads radially
- z = 0, becuase of the ground level
- h is the height of the chimney

In [76]:
def calculateCxyz(row, index):
    Cxyz = C_xyz(Q=row['PM10micro'],
                 u=row['sfcWindAVG'],
#                  sigmaY = 300,
                 sigmaY = row['sigmaY' + str(index)], 
#                  sigmaZ = 300,
                 sigmaZ = row['sigmaZ' + str(index)],
                 h = df_industrial.loc[index]['m'],
                 z = 0, 
                 y = row['y' + str(index)])
    return Cxyz

df_ind_model['Ctotal'] = 0
 
for index in range(0, df_industrial.shape[0]):
    df_hh_model['C' + str(index)] = df_hh_model.apply(lambda row: calculateCxyz(row, index), axis=1)
    df_hh_model['Ctotal'] = df_hh_model['Ctotal'] + df_ind_model['C' + str(index)]

In [77]:
df_ind_model

,Date,Station,PM10,PCS,HGHT(m),TEMP(C),TEMP(K),t/z,a,c<1km,...,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70
0,2016-11-01,STA-BG0052A,692.880,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,1.408322e-06,0.000837,6.001781e-04,0.000687,0.000678,9.044566e-05,0.000318,0.000318,0.001008,0.001017
1,2016-11-01,STA-BG0050A,823.440,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,1.673836e-05,0.000633,1.624575e-03,0.001884,0.001878,2.753297e-04,0.001212,0.001211,0.000393,0.000397
2,2016-11-01,STA-BG0073A,624.000,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,9.866119e-07,0.000337,6.499269e-04,0.000738,0.000734,6.060122e-04,0.002212,0.002207,0.000240,0.000242
3,2016-11-01,STA-BG0040A,876.240,E,1823,-1.4,276.6,0.003299,50.5,22.8,...,1.338528e-04,0.001019,-2.335282e-172,-0.000021,-0.000010,3.518470e-06,0.001925,0.001917,0.000714,0.000734
4,2016-11-02,STA-BG0052A,1632.960,E,1503,10.0,283.0,0.002257,50.5,22.8,...,7.112345e-06,0.004227,3.031035e-03,0.003471,0.003423,4.567710e-04,0.001605,0.001604,0.005092,0.005135
5,2016-11-02,STA-BG0050A,1756.560,E,1503,10.0,283.0,0.002257,50.5,22.8,...,7.651334e-05,0.002891,7.426157e-03,0.008610,0.008582,1.258570e-03,0.005542,0.005535,0.001795,0.001815
6,2016-11-02,STA-BG0073A,1516.560,E,1503,10.0,283.0,0.002257,50.5,22.8,...,5.138242e-06,0.001755,3.384798e-03,0.003846,0.003825,3.156092e-03,0.011518,0.011491,0.001249,0.001262
7,2016-11-02,STA-BG0040A,2382.288,E,1503,10.0,283.0,0.002257,50.5,22.8,...,7.798158e-04,0.005938,-1.360516e-171,-0.000122,-0.000056,2.049832e-05,0.011212,0.011167,0.004157,0.004278
8,2016-11-03,STA-BG0052A,953.280,E,1487,13.2,280.0,0.003618,50.5,22.8,...,1.614668e-06,0.000960,6.881155e-04,0.000788,0.000777,1.036977e-04,0.000364,0.000364,0.001156,0.001166
9,2016-11-03,STA-BG0050A,978.480,E,1487,13.2,280.0,0.003618,50.5,22.8,...,1.657493e-05,0.000626,1.608713e-03,0.001865,0.001859,2.726414e-04,0.001201,0.001199,0.000389,0.000393


In [ ]:
df = df_ind_model

## Normalizing from 0 to 1 (could also do from -1 to 1)

In [ ]:
df['HGHT(m)'] = (df['HGHT(m)'] - df['HGHT(m)'].min()) / (df['HGHT(m)'].max() - df['HGHT(m)'].min())

In [ ]:
df['HGHT(m)']